# DataFlow

## LungStage

In [1]:
from my_happy_graphviz import pydot
from my_happy_jupyter_utils import (
    image_utils
)


G = pydot.Dot(graph_type='digraph')

nm_data_node_xml = G.make_node('Nuclear Medicine Records (XML)', 'tab')
nm_data_node_xls = G.make_node('Nuclear Medicine Records (XLS)', 'tab')
nm_data_node_adb = G.make_node('Nuclear Medicine Records (AccessDB)', 'tab')
nm_data_node = G.make_node('Aggregate NM Data', 'trapezium')

G.make_link(nm_data_node_xml, nm_data_node, style='dashed')
G.make_link(nm_data_node_xls, nm_data_node, style='dashed')
G.make_link(nm_data_node_adb, nm_data_node, style='dashed')


onco_data_node = G.make_node('Oncology Records (XLS)', 'tab')
join_node = G.make_node('Merge Records', 'trapezium')
G.make_link(onco_data_node, join_node)
G.make_link(nm_data_node, join_node, 'Filter on disease\ncase and type')


pacs_node = G.make_node('PACS Records', 'cds')
solr_node = G.make_node('PACSCrawler/SOLR Database', 'box')
pacs_join_node = G.make_node('Match PET Scans', 'trapezium')

G.make_link(pacs_node, solr_node, 'pypacscrawler\nmetd', style='dashed')
G.make_link(solr_node, pacs_join_node, style='dashed')
G.make_link(join_node, pacs_join_node, style='dashed')


master_list_node = G.make_node('Save as full_list.json', 'tab')
G.make_link(pacs_join_node, master_list_node, 'Master Patient List')


screened_list_node = G.make_node('Screened List', 'box')
G.make_link(
    pacs_join_node, 
    screened_list_node, 
    'LungStage Screen\nGoogle Doc\n(Gregor)\n5min/patient')


download_node = G.make_node('Download for Annotation', 'box3d')
G.make_link(screened_list_node, download_node, style='dashed')


pet_download_node = G.make_node('Region Annotations', 'box3d')
G.make_link(
    download_node, 
    pet_download_node,
    'LungStage Annotation\nSlicer-based To ol\n(Alex and Thomas)\n30min/patient',
    style='dashed'
)

G.make_link(
    pet_download_node,
    screened_list_node,
    'Filter already\nannotated cases',
    style='dashed'
)

region_list_node = G.make_node('Region Data\nlsa.npz\nlsa.json', 'tab')
G.make_link(
    pet_download_node,
    region_list_node,
    'Save Region Data',
    style='dashed'
)

onco_list_node = G.make_node('Oncology List', 'box')
G.make_link(region_list_node, onco_list_node, style='dashed')
G.make_link(master_list_node, onco_list_node, style='dashed')



full_list_node = G.make_node('Full Patient Data', 'box')
G.make_link(
    onco_list_node,
    full_list_node,
    'LungStage Oncology Tool\nGoogle Docs\n(Audrey)\n30min/patient'
)

G.set_overlap(False)
G.set_rankdir('UD')

print(G.to_string())

file_name = '/'.join([
    'data/output/images',
    '0502_0101_lungstage.svg'
])

G.draw(file_name, prog='dot')

image_utils.show_image_with_title_by_url({
    'file_path': file_name,
    'title': 'LungStage',
})

digraph G {
overlap=False;
rankdir=UD;
"Nuclear Medicine Records (XML)" [shape=tab];
"Nuclear Medicine Records (XLS)" [shape=tab];
"Nuclear Medicine Records (AccessDB)" [shape=tab];
"Aggregate NM Data" [shape=trapezium];
"Nuclear Medicine Records (XML)" -> "Aggregate NM Data"  [penwidth=1, style=dashed];
"Nuclear Medicine Records (XLS)" -> "Aggregate NM Data"  [penwidth=1, style=dashed];
"Nuclear Medicine Records (AccessDB)" -> "Aggregate NM Data"  [penwidth=1, style=dashed];
"Oncology Records (XLS)" [shape=tab];
"Merge Records" [shape=trapezium];
"Oncology Records (XLS)" -> "Merge Records"  [penwidth=1, style=solid];
"Aggregate NM Data" -> "Merge Records"  [label="Filter on disease\ncase and type", penwidth=1, style=solid];
"PACS Records" [shape=cds];
"PACSCrawler/SOLR Database" [shape=box];
"Match PET Scans" [shape=trapezium];
"PACS Records" -> "PACSCrawler/SOLR Database"  [label="pypacscrawler\nmetd", penwidth=1, style=dashed];
"PACSCrawler/SOLR Database" -> "Match PET Scans"  [penw

# mri

In [2]:
from my_happy_graphviz import pydot
from my_happy_jupyter_utils import (
    image_utils
)


G = pydot.Dot(graph_type='digraph')

mri_node = G.make_node('mri', 'folder')
mri_node.set_label('MRI Image Data\n(T2, DWI Scout)')


sort_node = G.make_node('sort', 'record')
sort_node.set_label('{{Neural Image Sorter}|{DWI|T1|T2|Other}}')

G.make_link(mri_node, sort_node)


pos_node = G.make_node('position', 'record')
pos_node.set_label('{{Neural Position\nEstimator}|{z}}')

G.make_link(mri_node, pos_node)


stage_node = G.make_node('t2_stage', 'record')
stage_node.set_label(
    '{{Staging}|{T0|T1|T2|T3|T4}|{N0|N1|N2}|{M0|M1}}'
)

G.make_link(mri_node, stage_node, 'MRI Images')

G.make_link(sort_node, stage_node, 'MRI Category', style='dashed')
G.make_link(pos_node, stage_node, 'Position\nEstimate', style='dashed')


outcome_node = G.make_node('outcome', 'record')
outcome_node.set_label(
    '{{Outcome}|{Recurrence|Remission}}'
)

G.make_link(
    mri_node,
    outcome_node,
    'MRI Images'
)


G.make_link(stage_node, outcome_node, 'Stage\nEstimation', style='dashed')
G.make_link(sort_node, outcome_node, style='dashed')

for inode in [pos_node, sort_node]:
    inode.set_style('filled')
    inode.set_fillcolor('lightblue')
    

for inode in [stage_node, outcome_node]:
    inode.set_style('filled')
    inode.set_fillcolor('lightgreen')
    
    
G.set_rankdir('UD')


print(G.to_string())

file_name = '/'.join([
    'data/output/images',
    '0502_0201_mri.svg'
])

G.draw(file_name)

image_utils.show_image_with_title_by_url({
    'file_path': file_name,
    'title': 'mri',
})

digraph G {
rankdir=UD;
mri [label="MRI Image Data\n(T2, DWI Scout)", shape=folder];
sort [fillcolor=lightblue, label="{{Neural Image Sorter}|{DWI|T1|T2|Other}}", shape=record, style=filled];
mri -> sort  [penwidth=1, style=solid];
position [fillcolor=lightblue, label="{{Neural Position\nEstimator}|{z}}", shape=record, style=filled];
mri -> position  [penwidth=1, style=solid];
t2_stage [fillcolor=lightgreen, label="{{Staging}|{T0|T1|T2|T3|T4}|{N0|N1|N2}|{M0|M1}}", shape=record, style=filled];
mri -> t2_stage  [label="MRI Images", penwidth=1, style=solid];
sort -> t2_stage  [label="MRI Category", penwidth=1, style=dashed];
position -> t2_stage  [label="Position\nEstimate", penwidth=1, style=dashed];
outcome [fillcolor=lightgreen, label="{{Outcome}|{Recurrence|Remission}}", shape=record, style=filled];
mri -> outcome  [label="MRI Images", penwidth=1, style=solid];
t2_stage -> outcome  [label="Stage\nEstimation", penwidth=1, style=dashed];
sort -> outcome  [penwidth=1, style=dashed];
}

